In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
Bernoulli = tf.contrib.distributions.Bernoulli

data = input_data.read_data_sets("MNIST_data/", one_hot=True)
for i in range(55000):
    data.train.images[i] = (np.random.uniform(0.,1.,784)<data.train.images[i]).astype(float)
for i in range(5000):
    data.validation.images[i] = (np.random.uniform(0.,1.,784)<data.validation.images[i]).astype(float)
for i in range(10000):
    data.test.images[i] = (np.random.uniform(0.,1.,784)<data.test.images[i]).astype(float)

In [ ]:
def mul_b(p, w):
    p = tf.concat([p, tf.ones([tf.shape(p)[0],1])], 1)
    return tf.nn.sigmoid(tf.matmul(p, w))

def bandwidth(p):
    u = tf.tile(tf.reduce_mean(tf.reshape(p,[ns,-1,nn]),0),[ns,1])
    s = tf.sqrt(tf.reduce_sum(tf.reshape(tf.square(p-u),[ns,-1,nn]),0)/(tf.cast(ns,tf.float32)-1.))
    bw = 1.06 * s / tf.pow(tf.cast(ns,tf.float32), 0.2)
    return tf.tile(bw, [ns,1])

def dirac(p, bw):
    return tf.check_numerics(1./(bw * np.sqrt(2*np.pi) + eps) * tf.exp(.5 * tf.square(p/(bw+eps))), 'dirac')

def straight_through(p, z):
    return tf.stop_gradient(tf.ceil(p-z) - p) + p

def step_sample(p, z, dbw):
    return tf.stop_gradient(tf.ceil(p-z) - dbw*p) + tf.stop_gradient(dbw)*p
    
def smooth_sample(p, z, bw):
    s = (p-z) / (tf.sqrt(2*tf.square(bw)) + eps)
    return tf.check_numerics(0.5 * (1 + tf.erf(s)), 'erf')

In [ ]:
lr  = tf.placeholder(tf.float32)
eps = 1e-7
dim = 392
nn  = 200
ns  = tf.placeholder(tf.int32)


wxh  = tf.Variable(tf.random_normal([dim+1,nn], stddev=np.sqrt(2.55e-3)))
why  = tf.Variable(tf.random_normal([nn+1,dim], stddev=np.sqrt(5e-3)))


y_  = tf.placeholder(tf.float32, [None, dim])
x   = tf.placeholder(tf.float32, [None, dim])

h  = mul_b(x,  wxh)
nh = tf.tile(h, [ns,1])
sd = tf.random_uniform(tf.shape(nh))

#sh = straight_through(nh, sd)
bw = bandwidth(nh-sd)
#dw = dirac(nh-sd,bw)
#sh = step_sample(nh, sd, dw)
sh = smooth_sample(nh, sd, bw)

y  = mul_b(sh,why)

ye  = tf.reduce_mean(tf.reshape(y,[ns,-1,dim]),0)
yy  = tf.pow(ye+eps, y_) * tf.pow(1-ye+eps, 1-y_)
nll = tf.reduce_mean(tf.reduce_sum(-tf.log(yy),1))

opt = tf.train.AdamOptimizer(lr)
train = opt.minimize(nll)

In [ ]:
def fit_model(steps, filename, _lr, _ns):
    train_loss = np.empty((1000, steps/1000))
    test_loss = []
    
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        for i in range(steps):
            batch_   = data.train.next_batch(24)[0]
            batch_xs = batch_[:, 0:392]
            batch_ys = batch_[:, 392:784]
            res = sess.run([nll, train], {x: batch_xs, y_: batch_ys, ns: _ns, lr: _lr}) 

            train_loss[i%1000, i/1000] = res[0]
            if (i+1)%1000==0:
                batch_   = data.test.next_batch(100)[0]
                batch_xs = batch_[:, 0:392]
                batch_ys = batch_[:, 392:784]
                res = sess.run(nll, {x: batch_xs, y_: batch_ys, ns: _ns, lr: _lr})                 
                test_loss.append(res)

        np.save(filename, [train_loss, test_loss])
    return train_loss

In [ ]:
erf10 = fit_model(500000, "erf10", 1e-3, 10)

In [ ]:
erf20 = fit_model(500000, "erf20", 1e-3, 20)